In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
pos_cash = pd.read_csv('D:/Data group project DPV/POS_CASH_balance.csv')
app_train = pd.read_csv('D:/Data group project DPV/application_train.csv')

**POS_CASH_balance.csv**

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample.

**POS_CASH_balance.csv FEATURE DESCRIPTION**

SK_ID_PREV: The ID of previous credit in Home Credit related to loan in our sample

SK_ID_CURR: ID of loan in our sample

MONTHS_BALANCE: Month of balance relative to application date

CNT_INSTALMENT: Term of previous credit (can change over time)

CNT_INSTALMENT_FUTURE: Installments left to pay on the previous credit

NAME_CONTRACT_STATUS: Contract status during the month

SK_DPD: DPD (days past due) during the month of previous credit

SK_DPD_DEF: DPD during the month with tolerance (debts with low loan amounts are ignored) of the previous credit

In [8]:
pos_cash.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
5,2207092,342166,-32,12.0,12.0,Active,0,0
6,1110516,204376,-38,48.0,43.0,Active,0,0
7,1387235,153211,-35,36.0,36.0,Active,0,0
8,1220500,112740,-31,12.0,12.0,Active,0,0
9,2371489,274851,-32,24.0,16.0,Active,0,0


## **FEATURE ADDED**

In [9]:
pos_cash_new = pos_cash.groupby(['SK_ID_CURR']).last().reset_index()
pos_cash_new.rename(index=str,
                    columns={'CNT_INSTALMENT_FUTURE': 'pos_cash_remaining_installments'},
                    inplace=True)

In [10]:
pos_cash['is_contract_status_completed'] = pos_cash['NAME_CONTRACT_STATUS'] == 'Completed'
group_object = pos_cash.groupby(['SK_ID_CURR'])['is_contract_status_completed'].sum().reset_index()
group_object.rename(index=str,
                    columns={'is_contract_status_completed': 'pos_cash_completed_contracts'},
                    inplace=True)
pos_cash_new = pos_cash_new.merge(group_object, on=['SK_ID_CURR'], how='left')

In [11]:
pos_cash_new

,SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,pos_cash_remaining_installments,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,pos_cash_completed_contracts
0,100001,1369693,-56,4.0,3.0,Active,0,0,2
1,100002,1038818,-15,24.0,20.0,Active,0,0,0
2,100003,2636178,-28,6.0,6.0,Active,0,0,2
3,100004,1564014,-26,4.0,3.0,Active,0,0,1
4,100005,2495675,-15,9.0,0.0,Completed,0,0,1
...,...,...,...,...,...,...,...,...,...
337247,456251,2248017,-6,8.0,6.0,Active,0,0,1
337248,456252,1503599,-80,6.0,4.0,Active,0,0,1
337249,456253,2029307,-93,12.0,0.0,Completed,0,0,2
337250,456254,2016407,-2,16.0,9.0,Active,0,0,0


In [12]:
agg_pos_cash = pos_cash_new.groupby(['SK_ID_CURR'])['MONTHS_BALANCE', 'CNT_INSTALMENT',\
                                                  'pos_cash_remaining_installments',\
                                                   'pos_cash_completed_contracts',\
                                                   'SK_DPD', 'SK_DPD_DEF'].agg( ['min', 'max', 'mean'])

<ipython-input-12-effd92f8943f>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_pos_cash = pos_cash_new.groupby(['SK_ID_CURR'])['MONTHS_BALANCE', 'CNT_INSTALMENT',\


In [13]:
agg_pos_cash

MONTHS_BALANCE          CNT_INSTALMENT              \
                      min max mean            min   max  mean   
SK_ID_CURR                                                      
100001                -56 -56  -56            4.0   4.0   4.0   
100002                -15 -15  -15           24.0  24.0  24.0   
100003                -28 -28  -28            6.0   6.0   6.0   
100004                -26 -26  -26            4.0   4.0   4.0   
100005                -15 -15  -15            9.0   9.0   9.0   
...                   ...  ..  ...            ...   ...   ...   
456251                 -6  -6   -6            8.0   8.0   8.0   
456252                -80 -80  -80            6.0   6.0   6.0   
456253                -93 -93  -93           12.0  12.0  12.0   
456254                 -2  -2   -2           16.0  16.0  16.0   
456255                 -8  -8   -8           12.0  12.0  12.0   

           pos_cash_remaining_installments              \
                                       min   max  mean   
SK_ID_CURR                                               
100001                                 3.0   3.0   3.0   
100002                                20.0  20.0  20.0   
100003                                 6.0   6.0   6.0   
100004                                 3.0   3.0   3.0   
100005                                 0.0   0.0   0.0   
...                                    ...   ...   ...   
456251                                 6.0   6.0   6.0   
456252                                 4.0   4.0   4.0   
456253                                 0.0   0.0   0.0   
456254                                 9.0   9.0   9.0   
456255                                 5.0   5.0   5.0   

           pos_cash_completed_contracts          SK_DPD          SK_DPD_DEF  \
                                    min max mean    min max mean        min   
SK_ID_CURR                                                                    
100001                                2   2    2      0   0    0          0   
100002                                0   0    0      0   0    0          0   
100003                                2   2    2      0   0    0          0   
100004                                1   1    1      0   0    0          0   
100005                                1   1    1      0   0    0          0   
...                                 ...  ..  ...    ...  ..  ...        ...   
456251                                1   1    1      0   0    0          0   
456252                                1   1    1      0   0    0          0   
456253                                2   2    2      0   0    0          0   
456254                                0   0    0      0   0    0          0   
456255                                6   6    6      0   0    0          0   

                     
           max mean  
SK_ID_CURR           
100001       0    0  
100002       0    0  
100003       0    0  
100004       0    0  
100005       0    0  
...         ..  ...  
456251       0    0  
456252       0    0  
456253       0    0  
456254       0    0  
456255       0    0  

[337252 rows x 18 columns]

In [14]:
agg_pos_cash.columns = ['POS_CAS_' + c[0] + '_' + c[1].upper() for c in agg_pos_cash.columns.values.tolist()]

In [15]:
agg_pos_cash

,POS_CAS_MONTHS_BALANCE_MIN,POS_CAS_MONTHS_BALANCE_MAX,POS_CAS_MONTHS_BALANCE_MEAN,POS_CAS_CNT_INSTALMENT_MIN,POS_CAS_CNT_INSTALMENT_MAX,POS_CAS_CNT_INSTALMENT_MEAN,POS_CAS_pos_cash_remaining_installments_MIN,POS_CAS_pos_cash_remaining_installments_MAX,POS_CAS_pos_cash_remaining_installments_MEAN,POS_CAS_pos_cash_completed_contracts_MIN,POS_CAS_pos_cash_completed_contracts_MAX,POS_CAS_pos_cash_completed_contracts_MEAN,POS_CAS_SK_DPD_MIN,POS_CAS_SK_DPD_MAX,POS_CAS_SK_DPD_MEAN,POS_CAS_SK_DPD_DEF_MIN,POS_CAS_SK_DPD_DEF_MAX,POS_CAS_SK_DPD_DEF_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,
100001,-56,-56,-56,4.0,4.0,4.0,3.0,3.0,3.0,2,2,2,0,0,0,0,0,0
100002,-15,-15,-15,24.0,24.0,24.0,20.0,20.0,20.0,0,0,0,0,0,0,0,0,0
100003,-28,-28,-28,6.0,6.0,6.0,6.0,6.0,6.0,2,2,2,0,0,0,0,0,0
100004,-26,-26,-26,4.0,4.0,4.0,3.0,3.0,3.0,1,1,1,0,0,0,0,0,0
100005,-15,-15,-15,9.0,9.0,9.0,0.0,0.0,0.0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,-6,-6,-6,8.0,8.0,8.0,6.0,6.0,6.0,1,1,1,0,0,0,0,0,0
456252,-80,-80,-80,6.0,6.0,6.0,4.0,4.0,4.0,1,1,1,0,0,0,0,0,0
456253,-93,-93,-93,12.0,12.0,12.0,0.0,0.0,0.0,2,2,2,0,0,0,0,0,0


In [16]:
agg_pos_cash.isna().sum()

POS_CAS_MONTHS_BALANCE_MIN                       0
POS_CAS_MONTHS_BALANCE_MAX                       0
POS_CAS_MONTHS_BALANCE_MEAN                      0
POS_CAS_CNT_INSTALMENT_MIN                      28
POS_CAS_CNT_INSTALMENT_MAX                      28
POS_CAS_CNT_INSTALMENT_MEAN                     28
POS_CAS_pos_cash_remaining_installments_MIN     28
POS_CAS_pos_cash_remaining_installments_MAX     28
POS_CAS_pos_cash_remaining_installments_MEAN    28
POS_CAS_pos_cash_completed_contracts_MIN         0
POS_CAS_pos_cash_completed_contracts_MAX         0
POS_CAS_pos_cash_completed_contracts_MEAN        0
POS_CAS_SK_DPD_MIN                               0
POS_CAS_SK_DPD_MAX                               0
POS_CAS_SK_DPD_MEAN                              0
POS_CAS_SK_DPD_DEF_MIN                           0
POS_CAS_SK_DPD_DEF_MAX                           0
POS_CAS_SK_DPD_DEF_MEAN                          0
dtype: int64

In [17]:
agg_pos_cash.fillna(0, inplace = True)
agg_pos_cash.reset_index(inplace = True)

In [18]:
agg_pos_cash
# POS_CASH_balance.csv

,SK_ID_CURR,POS_CAS_MONTHS_BALANCE_MIN,POS_CAS_MONTHS_BALANCE_MAX,POS_CAS_MONTHS_BALANCE_MEAN,POS_CAS_CNT_INSTALMENT_MIN,POS_CAS_CNT_INSTALMENT_MAX,POS_CAS_CNT_INSTALMENT_MEAN,POS_CAS_pos_cash_remaining_installments_MIN,POS_CAS_pos_cash_remaining_installments_MAX,POS_CAS_pos_cash_remaining_installments_MEAN,POS_CAS_pos_cash_completed_contracts_MIN,POS_CAS_pos_cash_completed_contracts_MAX,POS_CAS_pos_cash_completed_contracts_MEAN,POS_CAS_SK_DPD_MIN,POS_CAS_SK_DPD_MAX,POS_CAS_SK_DPD_MEAN,POS_CAS_SK_DPD_DEF_MIN,POS_CAS_SK_DPD_DEF_MAX,POS_CAS_SK_DPD_DEF_MEAN
0,100001,-56,-56,-56,4.0,4.0,4.0,3.0,3.0,3.0,2,2,2,0,0,0,0,0,0
1,100002,-15,-15,-15,24.0,24.0,24.0,20.0,20.0,20.0,0,0,0,0,0,0,0,0,0
2,100003,-28,-28,-28,6.0,6.0,6.0,6.0,6.0,6.0,2,2,2,0,0,0,0,0,0
3,100004,-26,-26,-26,4.0,4.0,4.0,3.0,3.0,3.0,1,1,1,0,0,0,0,0,0
4,100005,-15,-15,-15,9.0,9.0,9.0,0.0,0.0,0.0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337247,456251,-6,-6,-6,8.0,8.0,8.0,6.0,6.0,6.0,1,1,1,0,0,0,0,0,0
337248,456252,-80,-80,-80,6.0,6.0,6.0,4.0,4.0,4.0,1,1,1,0,0,0,0,0,0
337249,456253,-93,-93,-93,12.0,12.0,12.0,0.0,0.0,0.0,2,2,2,0,0,0,0,0,0
337250,456254,-2,-2,-2,16.0,16.0,16.0,9.0,9.0,9.0,0,0,0,0,0,0,0,0,0


### **Những feature được tạo thêm**

**pos_cash_remaining_installments:** số khoản vay còn phải trả

**pos_cash_completed_contracts:** số khoản vay đã được hoàn thành

### **Dấu hiệu nhận biết bảng POS_CASH_balance.csv: POS_CAS_**

In [19]:
agg_pos_cash.to_csv('C:/Users/Admin/Data prepare and Visualization/Group project/FINAL_POS_CASH_balance.csv')